# EDA Agent Experiment Loop

This notebook demonstrates the interaction between the **EDA Agent** and the **PanGu-Weather Inference** model.

### Setup
Ensure you have installed the requirements using `pip install -r ../weather/requirements.txt`.

In [ ]:
import sys
import os
import numpy as np
import matplotlib.pyplot as plt

# Add project root to path to import modules
sys.path.append(os.path.abspath('..'))

from weather.inference import PanGuInference
from eda.agent import EDA_Agent

### Initialize Components

In [ ]:
# Path to your ONNX model (Update this with actual path)
MODEL_PATH = "../weather/pangu_weather_24h.onnx"

# Initialize Inference Wrapper
# Note: If model is missing, it will warn but proceed with dummy data for testing logic
pangu = PanGuInference(MODEL_PATH, device='cpu')

# Initialize EDA Agent
agent = EDA_Agent(mode='simulation')

### Run Experiment Loop

In [ ]:
steps = 10
rewards = []

print(f"Starting loop for {steps} steps...")

for i in range(steps):
    # 1. Get weather observation (in simulation, just a random state or next step)
    if pangu.session:
        # Use model input shape if session active
        weather_observation = pangu.dummy_input()
    else:
        weather_observation = np.random.randn(1, 5, 13, 100, 100).astype(np.float32)

    # In reality, this would be the observable aspects of the weather predictor at t+1
    predictor_observation = pangu.predictor_observation()
    weather_prediction = pangu.predict(weather_observation)

    # 2. Agent Perceives & Acts
    # The agent might modify the input state (action) before inference,
    # or simply observe the result and update its policy.
    action, reward = agent.run_step(predictor_observation, weather_observation, weather_prediction)
    
    rewards.append(reward)
    
    if i % 2 == 0:
        print(f"Step {i}: Action={action}, Reward={reward:.4f}")

print("Experiment Complete.")

### Visualization

In [ ]:
plt.figure(figsize=(10, 5))
plt.plot(rewards, label='Negative MSE (Reward)')
plt.xlabel('Step')
plt.ylabel('Reward')
plt.title('EDA Agent Performance Over Time')
plt.legend()
plt.grid(True)
plt.show()